In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [82]:
df = {}
for year in range(1997,2018):
    df[year] = pd.read_csv('data/cleaned/msa_naicssector_cleaned_totEmploy_'+str(year)+'.csv')
    df[year]['Total Employment'] = pd.to_numeric( df[year]['Total Employment'],downcast='integer',errors='coerce' )
    df[year] = df[year].dropna()
    #df[year].rename(columns= {'Total Employment':'Total Employment - '+str(year)} ,inplace=True)
    
    del df[year]['FIPS Code'], df[year]['NAICS Description'], df[year]['Unnamed: 0']
    

In [4]:
unique_NAICS_codes = set( df[1998]['NAICS Code'].unique() )
for year in range(1999,2018):
    unique_NAICS_codes = unique_NAICS_codes & set( df[year]['NAICS Code'].unique() )

In [ ]:
#for year in range(1998,2018):
 #   df[year]['NAICS Code'] = df[year]['NAICS Code'].map(lambda x: x in unique_NAICS_codes)

In [5]:
MSA_df = pd.read_csv('data/MSA_wikipedia.csv',delimiter='\t')

In [72]:
MSAs = MSA_df['Metropolitan statistical area ']
MSA_shorts = MSAs.map(lambda x: x.split(',')[0].split('-')[0].lower() )
MSA_shorts_dict = {MSA_shorts.iloc[i]:
                   {'MSA': MSAs.iloc[i][:-1],
                    'State': MSAs.iloc[i].split(', ')[1].split(' ')[0] } for i in range(len(MSAs))  }

In [83]:
# for 1997-2002
def standard_MSA_v1(string):
    for short in MSA_shorts_dict.keys():
        if (short in string.lower()) and string.split(', ')[1].split(' ')[0]==MSA_shorts_dict[short]['State']:
            return MSA_shorts_dict[short]
    return { 'MSA':None, 'State':None }
# for 2003-2017
def standard_MSA_v2(string):
    if 'micro' in string.lower():
        return { 'MSA':None, 'State':None }
    for short in MSA_shorts_dict.keys():
        if (short in string.lower()) and string.split(', ')[1].split(' ')[0]==MSA_shorts_dict[short]['State']:
            return MSA_shorts_dict[short]
    return { 'MSA':None, 'State':None }

for year in range(1998,2003):
    df[year]['MSA'] = df[year]['MSA Description'].map( lambda x: standard_MSA_v1(x)['MSA'] )
    #df[year]['State'] = df[year]['MSA Description'].map( lambda x: standard_MSA_v1(x)[1] )
    df[year] = df[year][ ~df[year]['MSA'].isna() ] 
    del df[year]['MSA Description']
        
for year in range(2003,2018):
    df[year]['MSA'] = df[year]['MSA Description'].map( lambda x: standard_MSA_v2(x)['MSA'] )
    #df[year]['State'] = df[year]['MSA Description'].map( lambda x: standard_MSA_v2(x)[1] )
    df[year] = df[year][ ~df[year]['MSA'].isna() ] 
    del df[year]['MSA Description']
    

In [17]:
#df[2016]['MSA'].unique()
df[1998].to_csv('data/susb/msa_1998_draft_standardMSA.csv')

In [156]:
#for year in range(1998,2018):
 #   print( len(df[year]) )

In [201]:
df[1998]

,MSA Description,NAICS Code,Total Employment - 1998
0,"Abilene, TX",--,48503
1,"Abilene, TX",11,[a]
2,"Abilene, TX",21,1042
3,"Abilene, TX",22,536
4,"Abilene, TX",23,2396
...,...,...,...
7270,"Yuma, AZ",71,[f]
7271,"Yuma, AZ",72,4059
7272,"Yuma, AZ",81,[g]
7273,"Yuma, AZ",95,[b]


In [84]:
df1 = {}
for year in range(1998,2018):
    df1[year] = df[year].set_index(['MSA','NAICS Code'],drop=True)
    print( len(df1[year]) )

3828
3969
3968
3985
3950
4941
4956
4795
4760
4371
6438
6661
6666
6662
7316
7329
7333
7331
7331
7179


In [158]:
MSA = ''
NAICS_code = '--'

,Unnamed: 0,FIPS Code,NAICS Code,NAICS Description,Enterprise Employment Size,Total Employment,MSA,State
0,2,10100.0,--,Total,01: Total,19533.0,"(None, None)",None
1,11,10100.0,11,"Agriculture, Forestry, Fishing and Hunting",01: Total,20.0,"(None, None)",None
2,16,10100.0,21,"Mining, Quarrying, and Oil and Gas Extraction",01: Total,24.0,"(None, None)",None
3,21,10100.0,22,Utilities,01: Total,134.0,"(None, None)",None
4,28,10100.0,23,Construction,01: Total,856.0,"(None, None)",None
...,...,...,...,...,...,...,...,...
18964,149758,49820.0,56,Administrative and Support and Waste Managemen...,01: Total,95.0,"(None, None)",None
18965,149765,49820.0,62,Health Care and Social Assistance,01: Total,267.0,"(None, None)",None
18966,149774,49820.0,71,"Arts, Entertainment, and Recreation",01: Total,0.0,"(None, None)",None
18967,149778,49820.0,72,Accommodation and Food Services,01: Total,165.0,"(None, None)",None


In [77]:
#pd.concat( [df1[1999], df1[2000]], axis=1)
dff = pd.merge(df1[1999], df1[2000],how='inner',on=['MSA','NAICS Code'] )
#pd.merge(df1[1999], df1[2000],how='inner',left_index=True ,right_index=True)
len(dff)

4812

In [70]:
df[1999].iloc[:70].to_csv('data/susb/msa_1999_draft_merge.csv')
df[2000].iloc[:60].to_csv('data/susb/msa_2000_draft_merge.csv')
dff.to_csv('data/susb/msa_both_draft_merge.csv')

In [86]:
df_all = df1[1998]
for year in range(1999,2018):
    df_all = pd.merge(df_all, df1[year],how='inner' ,on=['MSA','NAICS Code'],copy=False )
    print(len(df_all) )

3693
3349
3276
3212
2839
2754
2684
2634
1751
1742
1742
1742
1742
1742
1742
1742
1742
1742
1742


In [26]:
df1[1999].duplicated()

MSA              NAICS Code
Abilene, TX MSA  --            False
                 11            False
                 21            False
                 22            False
                 23            False
                               ...  
Yuma, AZ MSA     71            False
                 72             True
                 81             True
                 95             True
                 99             True
Length: 6418, dtype: bool

In [87]:
df_all

Total Employment - 1998  Total Employment - 1999  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          48503.0                  48514.0   
                21                           1042.0                    491.0   
                23                           2396.0                   2300.0   
                31-33                        3462.0                   3702.0   
                44-45                        7342.0                   7339.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        3082.0                   2164.0   
                42                           1872.0                   1682.0   
                44-45                        6308.0                   6476.0   
                52                            934.0                    964.0   
                54                            717.0                    745.0   

                            Total Employment - 2000  Total Employment - 2001  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          49157.0                  48664.0   
                21                            635.0                    693.0   
                23                           2506.0                   2316.0   
                31-33                        3706.0                   3281.0   
                44-45                        7230.0                   7489.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        2321.0                   2317.0   
                42                           1626.0                   1814.0   
                44-45                        6681.0                   7076.0   
                52                            951.0                    970.0   
                54                            804.0                    863.0   

                            Total Employment - 2002  Total Employment - 2003  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          49969.0                  57108.0   
                21                            834.0                    875.0   
                23                           3002.0                   3670.0   
                31-33                        3177.0                   3487.0   
                44-45                        7633.0                   8472.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        2856.0                   3111.0   
                42                           3686.0                   1880.0   
                44-45                        6558.0                   6675.0   
                52                            859.0                    898.0   
                54                            974.0                   1037.0   

                            Total Employment - 2004  Total Employment - 2005  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          55427.0                  53961.0   
                21                            931.0                   1038.0   
                23                           2537.0                   2909.0   
                31-33                        2918.0                   2792.0   
                44-45                        8634.0                   8586.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        3398.0                   3621.0   
                42                           1854.0                   1749.0   
                44-45                        7019.0                   8316.0   
                52       

In [89]:
for year in range(1998,2017):
    df_all['TotEmp - Percentage Point Change - '+str(year)+'-'+str(year+1)] = \
    (df_all['Total Employment - '+str(year+1)]- df_all['Total Employment - '+str(year)])/df_all['Total Employment - '+str(year)]

In [90]:
df_all

Total Employment - 1998  Total Employment - 1999  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          48503.0                  48514.0   
                21                           1042.0                    491.0   
                23                           2396.0                   2300.0   
                31-33                        3462.0                   3702.0   
                44-45                        7342.0                   7339.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        3082.0                   2164.0   
                42                           1872.0                   1682.0   
                44-45                        6308.0                   6476.0   
                52                            934.0                    964.0   
                54                            717.0                    745.0   

                            Total Employment - 2000  Total Employment - 2001  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          49157.0                  48664.0   
                21                            635.0                    693.0   
                23                           2506.0                   2316.0   
                31-33                        3706.0                   3281.0   
                44-45                        7230.0                   7489.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        2321.0                   2317.0   
                42                           1626.0                   1814.0   
                44-45                        6681.0                   7076.0   
                52                            951.0                    970.0   
                54                            804.0                    863.0   

                            Total Employment - 2002  Total Employment - 2003  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          49969.0                  57108.0   
                21                            834.0                    875.0   
                23                           3002.0                   3670.0   
                31-33                        3177.0                   3487.0   
                44-45                        7633.0                   8472.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        2856.0                   3111.0   
                42                           3686.0                   1880.0   
                44-45                        6558.0                   6675.0   
                52                            859.0                    898.0   
                54                            974.0                   1037.0   

                            Total Employment - 2004  Total Employment - 2005  \
MSA             NAICS Code                                                     
Abilene, TX MSA --                          55427.0                  53961.0   
                21                            931.0                   1038.0   
                23                           2537.0                   2909.0   
                31-33                        2918.0                   2792.0   
                44-45                        8634.0                   8586.0   
...                                             ...                      ...   
Yuma, AZ MSA    31-33                        3398.0                   3621.0   
                42                           1854.0                   1749.0   
                44-45                        7019.0                   8316.0   
                52       

In [3]:
df11 = pd.DataFrame(dict(A=[1, 1, 1], B=[8, 8, 7]))
df12 = pd.DataFrame(dict(A=[1, 2, 1], B=[8, 8, 4]))
df11,df12

(   A  B
 0  1  8
 1  1  8
 2  1  7,
    A  B
 0  1  8
 1  2  8
 2  1  4)

In [5]:
df11.rename( columns={'A':pd.MultiIndex.from_tuples( ('A','a') ) } )

TypeError: unhashable type: 'MultiIndex'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
  File "/usr/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 4273, in __hash__
    raise TypeError(f"unhashable type: {repr(type(self).__name__)}")
TypeError: unhashable type: 'MultiIndex'


TypeError: unhashable type: 'MultiIndex'

TypeError: unhashable type: 'MultiIndex'

In [28]:
pd.merge( df11,df12,how='inner',left_index=True,right_index=True )
pd.merge( df11,df12,how='inner',left_index=True,right_index=True )

,A_x,B,A_y,C
0,1,9,1,6
1,1,8,1,5
2,1,7,1,4


In [43]:
df111 = df11.set_index( [df11.index, 'B'] )
df121 = df12.set_index( [df12.index, 'B'] )
df111,df121

(     A
   B   
 0 8  1
 1 8  1
 2 7  1,
      A
   B   
 0 8  1
 1 8  2
 2 4  1)

In [44]:
pd.merge( df111,df121,how='inner',left_index=True,right_index=True )

,,A_x,A_y
,B,,
0,8,1,1
1,8,1,2
